# Introdução ao Reconhecimento de Padrões, 2020.2, UFC/DETI
## Trabalho 2

Aluno : Thyago Freitas da Silva <br>
Matrícula : 392035

In [11]:
import numpy as np
import operator
import collections
import pandas as pd
from random import randrange
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

### Funções úteis durante o decorrer do relatório

#### Calcular média de todas as colunas de uma matriz e subtrair a média das respectivas colunas

In [12]:
def remove_median(data):
    columns = data.columns.values
    for c in columns:
        mean = data[c].mean()
        data[c] = data[c].apply(lambda value : value - mean)
    return data

#### Calcular matriz de covariância de uma matriz

In [13]:
def calc_cov(data):
    return np.cov(data, rowvar = False)

#### Calcular autovalores e autovetores

In [14]:
def calc_eigen_values_and_vectors(data):
    return np.linalg.eig(data)

#### Calcular mediana

In [15]:
def calc_median(array):
    values = []
    for v in array:
        if v != '?':
            v = int(v)
            values.append(v)
    return np.median(values)

#### Normalizar dados

In [16]:
def normalize(data):
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(data))

#### Encontrar o número Q com base no percentual exigido

In [17]:
def chooseQ(eigen_values,eigen_vectors,percentual):
    total = 0
    Q = 0
    variance = []
    for index in range(len(eigen_values)):
        value = eigen_values[index]/np.sum(eigen_values)
        variance.append(value)
    for v in variance:
        total += v
        Q += 1
        if total >= percentual:
            break
    return Q,variance

## Parte 0 : Banco de dados - Demartology

O banco de dados utilizado foi o "Demartology" que possui as seguintes características:

<ul>
<li>O arquivo CSV com os dados possui 35 colunas, sendo que a última coluna representa de forma numérica uma doenção demartológica.</li>
<li>A coluna 33 (band-like infiltrate) apresenta valores inválidos e que precisam ser removidos ou processados.</li>
<li>Temos 366 amostras no banco de dados.</li>
<li>A coluna 35 representa uma doença demartológica de forma numérica(1 à 6), sendo elas :</li>
    <ol>
        <li>psoriasis</li>
        <li>seboreic dermatitis</li>
        <li>lichen planus</li>
        <li>pityriasis rosea</li>
        <li>cronic dermatitis</li>
        <li>pityriasis rubra pilaris</li>
    </ol>
</ul>


In [18]:
data = pd.read_csv("./data/dermatology.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


## Parte 1 : Principal Component Analysis
### Funcionamento do algoritmo

O algoritmo conhecido como Principal Component Analysis, abreviado como PCA, é uma técnica de análise multivariada que costuma ser utilizada para analisar a relação entre features de um conjunto de amostras a fim de detectar uma forma de reduzir a dimensionalidade das amostras através da diminuição da redundância nos dados ,identificando variáveis que possuem grau considerável de correlação e as condensando, com perda mínima de informação, em um conjunto menor de componentes.

Abaixo temos os passos necessários para calcular o PCA.

<ol>
<li>Padronizar features de forma que todas as features sejam transformadas para a mesma ordem de grandeza.</li>
<li>Calcular matriz de covariância com base nos dados modificados pelo passo 1.</li>
<li>Calcular autovetores e autovalores.</li>
<li>Escolher o número Q que representa a quantidade de autovetores.</li>
<li>Multiplicar os autovetores escolhidos pela matriz resultante do passo 1.</li>
</ol>

O artigo ['Feature selection using principal component analysis'](https://ieeexplore.ieee.org/document/5640135) propõe uma heurística para que o PCA faça feature selection, ou seja, seja capaz de designar quais das features do conjunto de dados original podem ser utilizadas sem perda de informação e quais podem ser descartadas. Os passos necessários seguem abaixo:


<ol>
<li>Ao calcular os autovetores e autovalores, guardar os indexes originais que associam cada autovalor a uma feature do conjunto de dados inicial.</li>
<li>Ao selecionar os Q componentes mais importantes, selecionar dentre os indexes guardados no passo 1, quais estão associados aos Q componentes, de forma que cada index indica qual feature do conjunto de dados original deve ser selecionada.</li>
</ol>

### Implementacão do Principal Component Analysis

In [19]:
class PCA:
    def __init__(self, percentual):
        self.percentual = percentual
        self.vectors = []
        self.Q = 0
    def fit_predict(self, data):
        cov_matrix = calc_cov(data)
        eigenvalues,eigenvectors = calc_eigen_values_and_vectors(cov_matrix)
        sorted_index = np.argsort(eigenvalues)[::-1]
        sorted_eigenvalue = eigenvalues[sorted_index]
        sorted_eigenvectors = eigenvectors[:,sorted_index]
        self.q ,self.variance = chooseQ(sorted_eigenvalue,sorted_eigenvectors,self.percentual)
        self.vectors = sorted_eigenvectors[:self.q] 
        self.values = sorted_eigenvalue
        self.features_selected = sorted_index[:self.q]

## Parte 1.1 : Principal Component Analysis
### Obtendo Q

#### Leitura da base "demartology"

In [20]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.head()

NameError: name 'pandas' is not defined

#### Pré-processamento

Como informado anteriormente, a coluna 33 do banco de dados possui valores inválidos que atrapalham a execução do algoritmo, logo, afim de evitar a remoção da amostras que possuem esse problema, foi adotado a heurística de substituir os valores inválidos,pontos de interrogação,pela mediana da coluna. A mediana foi escolhida por ser menos sensível a outliers se comparada com a média, por exemplo.

In [ ]:
median = calc_median(data.iloc[:,33])
data.iloc[:,33] = list(map(lambda value: median if value == '?' else value, data.iloc[:,33]))
data.iloc[:,33] = data.iloc[:,33].astype(np.int64)

Agora basta separar o conjunto de dados em features e as classes reais de cada amostra.

In [ ]:
normalized = normalize(data.iloc[:,:34])
features = normalized.iloc[:,:34]
classes = data.iloc[:,34]
features

#### Obtendo o valor Q para um percentual de 95%.

In [ ]:
pca = PCA(0.95)
pca.fit_predict(features)

In [ ]:
print(pca.q)

O valor obtido significa que podemos utilizar apenas 3 dimensões com os componentes obtidos, ao invés de utilizar as N dimensões iniciais.Os componentes estão salvos em pca.vectors

In [ ]:
pca.vectors

In [ ]:
sum_values = []
for i in range(len(pca.variance)):
    sum_values.append(pca.variance[i])
    if len(sum_values) > 1:
        sum_values[i] += sum_values[i-1]
    
plt.rcParams["figure.figsize"] = (10,5)
plt.bar(range(1,len(sum_values)+1),sum_values)
plt.title("Relação entre número de componentes principais e o percentual de variância associado")
plt.xlabel("Número de componentes principais")
plt.ylabel("Percentual 'explicado' pelos componentes")
plt.show()

Podemos ver que acima de 20 componentes principais, passamos a ter pouco acréscimo no percentual de variância, de forma que ao aumentar o número de componentes principais estariamos aumentando a complexidade sem ganhar de forma significante a capacidade de explicar os dados. Logo, podemos restringir o número de componentes à um número próximo de 20.

#### Multiplicar o conjunto de dados normalizado pelos componentes encontrados

In [ ]:
new_features_matrix = features.dot(np.transpose(pca.vectors))
new_features_matrix

### Comparando resultados do KNN com PCA e sem PCA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

#### Com PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_features_matrix, classes, test_size=0.2, random_state=1, stratify=classes)

knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train,y_train)
knn.predict(X_test)
score = knn.score(X_test, y_test)
print("Score with PCA : {:.2f}%".format(score*100))

#### Sem PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, classes, test_size=0.2, random_state=1, stratify=classes)

knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train,y_train)
knn.predict(X_test)
score = knn.score(X_test, y_test)
print("Score without PCA : {:.2f}%".format(score*100))

Podemos ver que ao aplicar o PCA para extrair features obtivemos um melhor resultado no KNN e graças ao fato de termos diminuido a dimensão do conjunto de dados, também diminuimos a complexidade de cálculos que o classificador precisa realizar.

## Realizando feature selection usando PCA

Dentro da variaveis features_selected, temos acesso as features originais selecionadas pelo PCA ao aplicar a heurística proposta no artigo comentado anteriormente.Logo, podemos usá-las para selecionar apenas as colunas originais necessárias.

#### Com as features selecionadas pelo PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features.iloc[:,pca.features_selected], classes, test_size=0.2, random_state=1, stratify=classes)

knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train,y_train)
knn.predict(X_test)
score = knn.score(X_test, y_test)
print("Score with PCA : {:.2f}%".format(score*100))

#### Sem as colunas selecionadas pelo PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, classes, test_size=0.2, random_state=1, stratify=classes)

knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train,y_train)
knn.predict(X_test)
score = knn.score(X_test, y_test)
print("Score without PCA : {:.2f}%".format(score*100))

Podemos ver que ao aplicar o PCA para selecionar features obtivemos um resultado aproximadamente igual ao KNN usando todas as features do conjunto de dados. Dessa forma, temos que diminuir a dimensionalidade do conjunto de dados não afeta o resultado do classificador, porém melhora o desempenho com relação a velocidade,pois trabalhamos com uma matriz com dimensões muito menores que a matriz original,diminuindo assim a quantidade de operações que devem ser realizadas pelo algoritmo.